# Training YOLOv8 - Python API

For comparison with launching the training from terminal (`yolo task=detect [...]`)


## Training with AMP

AMP = "Automatic Mixed Precision"

In [ ]:
%pip install -U ultralytics pillow

In [1]:
import os
import time
import sys
import ultralytics
from IPython import display
import torch

display.clear_output()

ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.8 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 5934MiB)
Setup complete ✅ (12 CPUs, 47.0 GB RAM, 124.0/456.9 GB disk)


In [3]:
# Training parameters:
n_epochs = 200
n_patience = 80
batch_size = 4
amp_flag = True

HOME_tomato_leaves_segm = os.getcwd()
print("Home: ", HOME_tomato_leaves_segm)

torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False
torch.backends.cudnn.enabled = True

cudnn_flg = torch.backends.cudnn.is_available()
print("cuDNN available: ", cudnn_flg)
if cudnn_flg:
    print("cuDNN version: ", torch.backends.cudnn.version())

Home:  /home/jovyan/projects/tomato-leaves-healthy-unhealthy
cuDNN available:  True
cuDNN version:  8902


### Model selection

Available YOLOv8 models (from least number of parameters to highest):

* YOLOv8n -> Nano
* YOLOv8s -> Small
* YOLOv8m -> Medium
* YOLOv8l -> Large
* YOLOv8x -> Extra-large

Notice that with bigger models not only the training takes much more time, but also inference does - meaning that, especially on a low-power device as the OAK-D lite, deploying the bigger models may be problematic.

In [4]:
## Select model:
# yolo_model = 'yolov8n.pt'  # Use YOLOv8 'Nano'
# yolo_model = 'yolov8s.pt'  # Use YOLOv8 'Small'
yolo_model = 'yolov8m.pt'  # Use YOLOv8 'Medium'
# yolo_model = 'yolov8l.pt'  # Use YOLOv8 'Large'
# yolo_model = 'yolov8x.pt'  # Use YOLOv8 'Extra-large'

In [ ]:
model = ultralytics.YOLO(yolo_model)
t_start = time.time()
results = model.train(data=os.path.join(HOME_tomato_leaves_segm, 'data.yaml'), epochs=n_epochs, patience=n_patience, batch=batch_size, imgsz=640, amp=amp_flag)
t_stop = time.time()

100%|██████████| 49.7M/49.7M [00:01<00:00, 43.4MB/s]
New https://pypi.org/project/ultralytics/8.0.207 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.8 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 5934MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/home/jovyan/projects/tomato-leaves-healthy-unhealthy/data.yaml, epochs=200, patience=80, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels

In [ ]:
print(f"Training time, AMP enabled: {t_stop - t_start} seconds")

### Validation

In [ ]:
model.val()